## This is my crawler for Amazon.com

### The crawler will built in python using;
- Functional Programming
- Object Oriented Programming

### First we import libraries for the job

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import time
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.common.by import By     
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options

### Then we create our header variable, This is for sites that return something different from what is supposed to be there, maybe as a result of client side languages or it was deliberate to ward off harmfull web crawlers.

In [2]:
HEADERS = ({'User-Agent':
        'Mozilla/5.0 (X11; Linux x86_64)\
        AppleWebKit/537.36 (KHTML, like Gecko)\
        Chrome/44.0.2403.157 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'})

## Scraping the site using functions

In [90]:
def amazon_product_details(url):
    amazon = {}
    driver = webdriver.Chrome(executable_path=r'C:\Users\Jchukwuedozi\Downloads\chromedriver_win32\chromedriver.exe')
    try:
        driver.get(url)
    except Exception as e:
        print(e)
    page_source = driver.page_source
    soup = bs(page_source)
    try:
        amazon['Title'] = soup.find(id='productTitle').get_text(' ', strip=True)
    except Exception:
        amazon['Title'] = None
    try:
        amazon['Price'] = soup.find(class_='a-color-price').get_text(' ', strip=True)
    except Exception:
        amazon['Price'] = None
    try:
        amazon['Review'] = soup.find(class_='a-icon-alt').get_text().split(' ')[0]
    except Exception:
        amazon['Review'] = None
    try:
        amazon['Rating'] = soup.find(id='acrCustomerReviewText').get_text().split(' ')[0]
    except Exception:
        amazon['Rating'] = None
    try:
        table = soup.find('table', class_='a-spacing-micro').find_all('tr')
    except Exception:
        print("a-spacing-micro can't be found")
    else:
        for row in table:
            content = row.find_all('td')
            key = content[0].get_text(' ', strip=True)
            value = content[1].get_text(' ', strip=True)
            amazon[key] = value
    
    try:
        product_details = soup.find('div', id='detailBullets_feature_div'
                ).find('ul', class_='a-unordered-list')
    except Exception:
        print("detailBullets_feature_div can't be found")
    else:
        for category in product_details.find_all('li'):
            product_key = [item.get_text(' ', strip=True).split('\n')[0] for item in category.find_all(class_='a-text-bold')]
            product_value = [item.find_next_sibling().get_text() for item in category.find_all(class_='a-text-bold')]
            for key, value in zip(product_key, product_value):
                amazon[key] = value
    return amazon    

In [89]:
def get_all_products(url, page_num):
    all_amazon_products = []
    driver = webdriver.Chrome(executable_path=r'C:\Users\Jchukwuedozi\Downloads\chromedriver_win32\chromedriver.exe')
    try:
        driver.get(url)
        page_source = driver.page_source
        soup = bs(page_source)
    except Exception as e:
        print(e)
    try:
        pages = int(soup.find_all('li', class_='a-disabled')[-1].get_text())
        for page in range(2):
            links = [i.get_attribute('href') for i in driver.find_elements_by_css_selector('.a-link-normal.s-no-outline')]
            for link in links:
                all_amazon_products.append(amazon_product_details(link))
            page_num +=1
            url = f'https://www.amazon.com/s?i=specialty-aps&rh=n%3A18505451011&fs=true&page={page_num}&qid=1625791103&ref=sr_pg_2'
            driver.get(url)
            page_source = driver.page_source
            soup = bs(page_source)
    except Exception as e:
        print(e)
    return all_amazon_products

In [88]:
products = get_all_products('https://www.amazon.com/s?i=specialty-aps&rh=n%3A18505451011&fs=true&page=2&qid=1625791103&ref=sr_pg_2', 2)

scraping... https://www.amazon.com/AXE-Shower-Tool-Detailer-Pack/dp/B0017TZD7S/ref=sr_1_17?dchild=1&qid=1626044199&sr=8-17
a-spacing-micro can't be found
scraping... https://www.amazon.com/Strip-Eyelash-Adhesive-Clear-0-176/dp/B01BMMOAFU/ref=sr_1_18?dchild=1&qid=1626044199&sr=8-18
scraping... https://www.amazon.com/Colgate-Extra-Clean-Toothbrush-Count/dp/B00CC6XSSQ/ref=sr_1_19?dchild=1&qid=1626044199&sr=8-19
scraping... https://www.amazon.com/CHI-Guard-Thermal-Protection-Spray/dp/B002RS6JSA/ref=sr_1_20?dchild=1&qid=1626044199&sr=8-20
scraping... https://www.amazon.com/CeraVe-Facial-Moisturizing-Lotion-AM/dp/B00F97FHAW/ref=sr_1_21?dchild=1&qid=1626044199&sr=8-21
scraping... https://www.amazon.com/Sensodyne-Pronamel-Whitening-Strengthening-Toothpaste/dp/B0762LYFKP/ref=sr_1_22?dchild=1&qid=1626044199&rdc=1&sr=8-22
scraping... https://www.amazon.com/Tweezerman-Stainless-Steel-Slant-Tweezer/dp/B000EMYJ88/ref=sr_1_23?dchild=1&qid=1626044199&sr=8-23
Message: unknown error: net::ERR_NAME_NOT_R

## Scraping the site with a class

In [91]:
# The class in use ------------------------------------------------------------
class Amaz:
    def __init__(self, url):
        self.url = url
        self.driver = webdriver.Chrome(executable_path=r'C:\Users\Jchukwuedozi\Downloads\chromedriver_win32\chromedriver.exe')
        
    def amazon_product_details(self, url):
        amazon = {}
        self.driver = webdriver.Chrome(executable_path=r'C:\Users\Jchukwuedozi\Downloads\chromedriver_win32\chromedriver.exe')
        try:
            self.driver.get(url)
        except Exception as e:
            print(e)
        self.page_source = self.driver.page_source
        self.soup = bs(self.page_source)
        try:
            amazon['Title'] = self.soup.find(id='productTitle').get_text(' ', strip=True)
        except Exception:
            amazon['Title'] = None
        try:
            amazon['Price'] = self.soup.find(class_='a-color-price').get_text(' ', strip=True)
        except Exception:
            amazon['Price'] = None
        try:
            amazon['Review'] = self.soup.find(class_='a-icon-alt').get_text().split(' ')[0]
        except Exception:
            amazon['Review'] = None
        try:
            amazon['Rating'] = self.soup.find(id='acrCustomerReviewText').get_text().split(' ')[0]
        except Exception:
            amazon['Rating'] = None
        try:
            table = self.soup.find('table', class_='a-spacing-micro').find_all('tr')
        except Exception:
            print("a-spacing-micro can't be found")
        else:
            for row in table:
                content = row.find_all('td')
                key = content[0].get_text(' ', strip=True)
                value = content[1].get_text(' ', strip=True)
                amazon[key] = value

        try:
            product_details = self.soup.find('div', id='detailBullets_feature_div'
                    ).find('ul', class_='a-unordered-list')
        except Exception:
            print("detailBullets_feature_div can't be found")
        else:
            for category in product_details.find_all('li'):
                product_key = [item.get_text(' ', strip=True).split('\n')[0] for item in category.find_all(class_='a-text-bold')]
                product_value = [item.find_next_sibling().get_text() for item in category.find_all(class_='a-text-bold')]
                for key, value in zip(product_key, product_value):
                    amazon[key] = value
        return amazon    
    
    
    def get_all_product(self, page_num):
        all_amazon_products = []
        self.page_num = page_num
        self.driver.get(self.url)
        self.page_source = self.driver.page_source
        self.soup = bs(self.page_source)
        pages = int(self.soup.find_all('li', class_='a-disabled')[-1].get_text())
        for page in range(pages): 
            links = [i.get_attribute('href') for i in self.driver.find_elements_by_css_selector('.a-link-normal.s-no-outline')]
            for link in links:
                all_amazon_products.append(self.amazon_products(link))
            self.page_num +=1    
            url = f'https://www.amazon.com/s?i=specialty-aps&rh=n%3A18505451011&fs=true&page={self.page_num}&qid=1625791103&ref=sr_pg_2'
            self.driver.get(url)
            self.page_source = self.driver.page_source
            self.soup = bs(self.page_source)
        return all_amazon_products